# Check the rationality of the molecular formula and revise it

In [467]:
ref_corr_df = pd.read_excel(f'{root_path}/ref_all-20240510-final.xlsx',sheet_name = "concat",index_col=0)
ref_corr_df.head() 

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,Corrected Formula,M.Wt,pos_neg,Adduct,refer_RT,m/z,RT time,NL,bscp
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,C11H15N5O4,281.27,pos,[M+H]+,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,C11H15N5O4,281.27,neg,[M-H]-,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,C21H30O4,346.46,pos,[M+H]+,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,C21H30O4,346.46,neg,[M-H]-,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,C12H18O2,194.27,neg,[M-H]-,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg


In [ ]:
for i in ref_corr_df.index:
    mol1 = ref_corr_df.loc[0, "Corrected Formula"]
    try:
        # Decompose molecular formula
        substance = Substance.from_formula(mol1)
        composition_dict = substance.composition
    except Exception as e:
        print(i)
        continue


In [466]:
composition_dict

{6: 11, 1: 15, 7: 5, 8: 4}

# Isotopes with a relative abundance of 100 are calculated
- Build a dictionary that stores all element qualities with a relative abundance of 100

In [698]:
import periodictable
from molmass import Formula
from periodictable import elements
from itertools import product
from chempy import Substance
import pandas as pd
import mzbatch,os

In [ ]:
def multiply_dicts(dict1, dict2):
    result = {}
    keys_union = set(dict1.keys()).union(dict2.keys())
    # print(keys_union)
    
    for key in keys_union:
        value1 = dict1.get(key, 0)  # Default value set to 0 to avoid affecting multiplication result
        value2 = dict2.get(key, 0)  # Default value set to 0 to avoid affecting multiplication result
        result[key] = value1 * value2

    return result


In [ ]:
abundacne100_dict = {}  # Record isotopes with relative abundance of 100
for i in range(1, 119):
    abundacne_dict = {}  # Record absolute abundance for each element {1: 99.9885, 2: 0.0115, 3: 0, 4: 0, 5: 0, 6: 0}
    elements = periodictable.elements[i]  # All isotopes of i
    for j in elements:
        abundacne_dict[j.isotope] = j.abundance
    max_value = max(abundacne_dict.values())
    if max_value == 0:
        continue
    normalized_dict = {key: value / max_value for key, value in abundacne_dict.items()}  # Convert absolute to relative
    key_value1 = next(key for key, value in normalized_dict.items() if value == 1.0)  # Get key with value of 1
    abundacne100_dict[i] = key_value1


In [ ]:
abund_mass_dict = {}
for i in abundacne100_dict.keys():
    elements = periodictable.elements[i]  # All isotopes of i
    for j in elements:
        if j.isotope == abundacne100_dict[i]:
            abund_mass_dict[i] = j.mass
# Add electron mass
abund_mass_dict[0] = Formula("H+").mass - Formula("H").mass


In [ ]:
# Convert [{6: 0}, {6: 1}], [{1: 0}, {1: 1}, {1: 2}], [{17: 0}, {17: 1}] to {6: 0, 1: 0, 17: 0}
def generate_combinations(*lists):
    return [dict(sum(map(list, map(dict.items, combination)), [])) for combination in product(*lists)]

# Filter valid combinations
# {6: 8, 1: 9, 0: 1} >= {6: 0, 1: 0, 0: 0}
def compare_dicts(dict1, dict2):
    # If any value in dict1 is less than dict2, return False
    for key in dict1:
        if dict1[key] < dict2[key]:
            return False
    return True


In [559]:
composition_no0_dict = {6: 12, 1: 27, 8: 12, 11: 1}
valence_dict = {key: valence_all_dict[key] for key, value in composition_no0_dict.items()}
valence1_dict = {key: value for key, value in valence_dict.items() if value == 1}
valencen_dict = {key: value for key, value in valence_dict.items() if value > 1}

valence1_sum = sum(composition_no0_dict[key] * valence1_dict.get(key, 0) for key in valence1_dict)
valencen_sum = sum(composition_no0_dict[key] * valencen_dict.get(key, 0) for key in valencen_dict)
valence1_dict, valence1_sum, valencen_dict, valencen_sum

({1: 1, 11: 1}, 28, {6: 4, 8: 2}, 72)

In [607]:
valence_all_dict = {1:1, 3:1, 4:2, 5:3, 6:4, 7:5, 8:2, 9:1,
                11:1, 12:3, 13:3, 14:4, 15:5, 16:4, 17:1, 19:1,
               20:2, 24:6, 25:7, 26:3, 27:3, 29:2,
                30:2,34:6,35:1,
                47:1,
                53:1,56:2,
                83:5}
# Enter: combinations_list
# Output: Return True if reasonable, False if not
# Note: H3C-NH-OH, calculation process: (4*1+5*1+2*1)-[2*(3-1)]+1=8, the first parenthesis is the total valency, the second parenthesis, the last +1 is the addition of H.
# Description: The n element can hold a position of 1 that is greater than the sum of the valence of 1
# Note: Taking into account NO2- ions, the highest setting of N is 5
def cal_mol_reasonable(composition_no0_dict):
    composition_no0_dict = {key: value for key, value in composition_no0_dict.items() if key != 0}
    
    valence_dict = {key: valence_all_dict[key] for key, value in composition_no0_dict.items()}
    valence1_dict = {key: value for key, value in valence_dict.items() if value == 1}
    valencen_dict = {key: value for key, value in valence_dict.items() if value > 1}
    
    valence1_sum = sum(composition_no0_dict[key] * valence1_dict.get(key, 0) for key in valence1_dict)
    valencen_sum = sum(composition_no0_dict[key] * valencen_dict.get(key, 0) for key in valencen_dict)
    valence1_sum, valencen_sum
    
    atomn_sum=sum([composition_no0_dict[key] for key, value in valencen_dict.items()])

    result = valencen_sum-2*(atomn_sum-1)+1
    if result>=valence1_sum:
        return True
    else:
        return False

cal_mol_reasonable({6: 12, 1: 27, 8: 12, 11: 1}) # C12H26O12Na +H(pos)
cal_mol_reasonable({6: 1, 1: 6, 7: 1, 8: 1}) # H3C-NH-OH +H(pos)
cal_mol_reasonable({6: 2, 1: 6}) # C2H6 +H

True

In [ ]:
def cal_mz_list(mol_formula, pos_neg):
    substance = Substance.from_formula(mol_formula)
    composition_dict = substance.composition
    print(f"Original molecular formula breakdown: {composition_dict}. Contains Na or K: {((11 in composition_dict.keys()) or (19 in composition_dict.keys()))}")
    
    if ((11 in composition_dict.keys()) or (19 in composition_dict.keys())):
        composition_dict[1] += 5
    print(f"After adding H5, molecular formula breakdown: {composition_dict}")
    
    # Ions in mass spectrometry gain or lose H, not electrons
    if pos_neg == "pos":
        electronic = 1
        composition_dict[1] = composition_dict[1] + 1
    else:
        electronic = -1
        composition_dict[1] = composition_dict[1] - 1
    print(composition_dict)
    
    if 0 in composition_dict:
        del composition_dict[0]  # Remove electrons
    
    single_2list = []
    for k, v in composition_dict.items():
        single_combinations = list(product(range(v + 1), repeat=1))  # Single combinations
        single_list = [dict(zip([k], i)) for i in single_combinations]
        single_2list.append(single_list)
    
    combinations_list = generate_combinations(*single_2list)
    print("All combinations:", combinations_list[-1])
    
    combinations_list = [i for i in combinations_list if cal_mol_reasonable(i)]

    combinations_list = [{**i, 0: electronic} for i in combinations_list]  # Combination list
    combinations_filter_list = [i for i in combinations_list if compare_dicts(composition_dict, i)]
    
    mz_list = [sum(multiply_dicts(abund_mass_dict, i).values()) for i in combinations_filter_list]
    mz_list = [x for x in mz_list if x >= 0]  # Filter for values greater than 0
    
    return mz_list


In [670]:
mz_list = cal_mz_list(mol_formula, pos_neg)
mz_list[-1]

原分子式拆解:{6: 25, 1: 51, 9: 3, 7: 1, 8: 6, 11: 1}。有无Na或K：True
增加H5分子式拆解:{6: 25, 1: 56, 9: 3, 7: 1, 8: 6, 11: 1}
{6: 25, 1: 57, 9: 3, 7: 1, 8: 6, 11: 1}
所有： {6: 25, 1: 57, 9: 3, 7: 1, 8: 6, 11: 1}


466.4102149554909

In [700]:
root_path = '../../Fulaoshi/230821_mzML/'
# middle_list = ['873_Amide','873_C18']
middle_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

# Calculate mz_list from the reference file

In [701]:
smi_ref_df = pd.read_excel(f'{root_path}/smi_ref_df.xlsx',index_col=0)
smi_ref_df['bscp']=smi_ref_df['batch']+'+'+smi_ref_df['sampleid'].astype(str)+'+'+smi_ref_df['CAS No.']+'+'+smi_ref_df['pos_neg']
smi_ref_df

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,MW,pos_neg,is_valid,mz,ref_rt,NL,bscp,batch_sampleid
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos,[M+H]+
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg,[M-H]-
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos,[M+H]+
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg,[M-H]-
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,neg,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg,[M-H]-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,700_C18,C18,S104,L-Allothreonine,28954-12-3,C4H9NO3,119.12,neg,1,118.05096,46.8,9.35E5,700_C18+S104+28954-12-3+neg,[M-H]-
13664,700_C18,C18,S104,3-O-Ethyl-L-ascorbic acid,86404-04-8,C8H12O6,204.18,neg,1,203.05611,307.8,9.02E5,700_C18+S104+86404-04-8+neg,[M-H]-
13665,700_C18,C18,S104,n6-(Aminoiminomethyl)lysine,yhh0927-1,C7H16N4O2,188.23,pos,1,189.13460,45.6,9.94E6,700_C18+S104+yhh0927-1+pos,[M+H]+
13666,700_C18,C18,S104,Galactosamine hydrochloride,1886979-58-3,C6H14ClNO5,215.63,pos,1,180.08664,43.8,2.15E7,700_C18+S104+1886979-58-3+pos,[M+H]+


In [685]:
bscp_list = smi_ref_df['bscp'].unique().tolist()

In [686]:
bscp_list[0:5]

['873_Amide+26386+1867-73-8+pos',
 '873_Amide+26386+1867-73-8+neg',
 '873_Amide+26386+50-22-6+pos',
 '873_Amide+26386+50-22-6+neg',
 '873_Amide+26386+136-77-6+neg']

In [ ]:
compare_mz_list = []  # For comparison with the provided data
mz_2list = []
for bscp in bscp_list[0:1]:
    print(bscp)
    
    iloc01 = smi_ref_df.columns.get_loc('Formula')
    iloc02 = smi_ref_df.columns.get_loc('pos_neg')
    temp_smi_ref_df = smi_ref_df[smi_ref_df['bscp'] == bscp].copy()
    mol_formula, pos_neg = temp_smi_ref_df.iloc[0, [iloc01, iloc02]]
    mol_formula = str(mol_formula).strip()
    
    # Handle calculation errors
    try:
        mz_list = cal_mz_list(mol_formula, pos_neg)
        # mz_2list.append(mz_list)
        # compare_mz_list.append(mz_list[-1])
        temp_smi_ref_df["cal_mz"] = mz_list[-1]
        temp_smi_ref_df["mz_list"] = [mz_list]
    except Exception as e:
        print("Error:", e)
        # mz_2list.append([])
        # compare_mz_list.append(-1)
        temp_smi_ref_df["cal_mz"] = -1
        temp_smi_ref_df["mz_list"] = [[]]
    
    temp_smi_ref_df.to_csv(f"output/smi_ref_df/{bscp}.csv")


873_Amide+26386+1867-73-8+pos
原分子式拆解:{6: 11, 1: 15, 7: 5, 8: 4}。有无Na或K：False
增加H5分子式拆解:{6: 11, 1: 15, 7: 5, 8: 4}
{6: 11, 1: 16, 7: 5, 8: 4}
所有： {6: 11, 1: 16, 7: 5, 8: 4}


In [ ]:
# Data check
for bscp in bscp_list:
    if not os.path.exists(f"output/smi_ref_df/{bscp}.csv"):
        print(bscp_list.index(bscp), bscp)
        # break


In [474]:
len(bscp_list)

13668

# Merge all data



In [690]:
all_files_list = [f"output/smi_ref_df/{bscp}.csv" for bscp in bscp_list]
merged_data_df = pd.DataFrame()
for file in all_files_list:
    df = pd.read_csv(file,index_col=0)
    merged_data_df = pd.concat([merged_data_df, df], ignore_index=True)
print(merged_data_df.shape)

(13668, 16)


In [696]:
# merged_data_df.to_excel('../../Fulaoshi/230821_mzML/smi_ref_df_mz_mzlist.xlsx',encoding='utf8')
# merged_data_df.to_csv('../../Fulaoshi/230821_mzML/smi_ref_df_mz_mzlist.csv',encoding='utf8')
merged_data_df.to_pickle('../../Fulaoshi/230821_mzML/smi_ref_df_mz_mzlist.pkl')

In [514]:
merged_data_df['diff'] = merged_data_df.apply(lambda x:mzbatch.cal_ppm(x['mz'],x['cal_mz']),axis=1)

In [515]:
merge_columns = ['batch', 'sepuzhu', 'sampleid', 'Product Name', 'CAS No.', 'Formula',
       'MW', 'pos_neg', 'is_valid', 'mz', 'ref_rt', 'NL', 'cal_mz','diff']

In [516]:
merged_data_df.loc[:,merge_columns].to_csv('../../Fulaoshi/230821_mzML/smi_ref_df_mz_diff.csv')

# Check whether the bscp is incomplete and the file is empty

In [ ]:
all_files_list = [f"output/smi_ref_df/{bscp}.csv" for bscp in bscp_list]

for file in all_files_list:
    if not os.path.exists(file):
        print(f"File does not exist: {file}")
        continue
    
    check_df = pd.read_csv(file, index_col=0)
    if check_df.shape[0] == 0:
        print(f"File is empty: {file}")


# Compare with the calculated m/z by Xcalibur 

In [ ]:
formula_mz_df = pd.read_excel("input/ion_mass_test.xlsx")


In [ ]:
test_mz_list = []
for i in formula_mz_df.index:
    mol_formula, pos_neg = formula_mz_df.loc[i, ['Molecular Formula', 'pos_neg']]
    mz_list = cal_mz_list(mol_formula, pos_neg)
    test_mz_list.append(mz_list[-1])


In [187]:
formula_mz_df['cal_mz'] = test_mz_list
formula_mz_df['diff'] = formula_mz_df['m/z'] - round(formula_mz_df['cal_mz'], 5)
formula_mz_df

,分子式,pos_neg,m/z,cal_mz,diff
0,C17H13Cl2F3N7O2S+,pos,506.01751,506.017510,0.00000
1,C17H12Cl2F3N7O2S+,pos,505.00968,505.009685,0.00000
2,C17H11Cl2F3N7O2S-,neg,504.00296,504.002957,0.00000
3,C62H113N11O12+2,pos,601.92796,1203.856470,-601.92851
4,C18H32O3Na+,pos,319.22437,319.224366,0.00000
5,C23H34O5Na+,pos,413.22985,413.229845,0.00000
6,C13H11BrNO3+,pos,307.99168,307.991682,0.00000
7,C25H53NO4P+,pos,462.37067,462.370672,0.00000
8,C21H15N2O2S-,neg,359.08597,359.085972,0.00000
9,C16H11O5-,neg,283.06120,283.061197,0.00000


# Check that the molecular formula is correct

In [467]:
ref_corr_df = pd.read_excel(f'{root_path}/ref_all-20240510-final.xlsx',sheet_name = "concat",index_col=0)
ref_corr_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,Corrected Formula,M.Wt,pos_neg,Adduct,refer_RT,m/z,RT time,NL,bscp
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,C11H15N5O4,281.27,pos,[M+H]+,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,C11H15N5O4,281.27,neg,[M-H]-,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,C21H30O4,346.46,pos,[M+H]+,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,C21H30O4,346.46,neg,[M-H]-,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,C12H18O2,194.27,neg,[M-H]-,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg


In [ ]:
for i in ref_corr_df.index:
    mol1 = ref_corr_df.loc[0,"Corrected Formula"]
    try:
        # Decompose molecular formula
        substance = Substance.from_formula(mol1)
        composition_dict = substance.composition
    except Exception as e:
        print(i)
        continue

In [466]:
composition_dict

{6: 11, 1: 15, 7: 5, 8: 4}

# 13 energy values that appear only once

# Calculate valid m/z values
- Merge the four graphs of 13 energy values
- Use only 39, removing those that appear only once, but keep all of 1 energy and 100 energy, as 1 energy might be fully fragmented, and 100 energy might just start fragmenting
- Fragment rationality: (mass_dif < -1) | (mass_dif <= 0.99999) | (3 <= mass_dif <= 13) | (21 <= mass_dif <= 24)
# Use relative abundance to calculate similarity


In [135]:
import pandas as pd
import numpy as np
import os,json,ast
import mzbatch
from sklearn.metrics.pairwise import cosine_similarity
import importlib as imp
imp.reload(mzbatch)

import warnings
warnings.filterwarnings('ignore')

### 设置路径

In [222]:
root_path = '../../Fulaoshi/230821_mzML'
# batch_list = ['873_Amide', '873_C18']
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

In [398]:
concat_info_all_df = pd.read_pickle(f'{out_root_path}/concat_MS2_info_all_df.pkl')

concat_info_all_df = concat_info_all_df[(concat_info_all_df['is_max_ms1']==1)|(concat_info_all_df['is_max_mol']==1)|(concat_info_all_df['is_max_base']==1)]
# Discard rows where filename is false, these are cases where there are no files at that particular energy
concat_info_all_df = concat_info_all_df[(concat_info_all_df['filename'].astype(str)!='False') & (concat_info_all_df['EV'].astype(str)!='204060')]

In [397]:
print(concat_info_all_df.shape)
concat_info_all_df.head()

(220436, 19)


,batch,sampleid,CAS No.,EV,pos_neg,mz,Product Name,filename,refer_valid,ref_rt,ref_intensity,is_max_ms1,ms1_intensity,is_max_mol,mol_intensity,is_max_base,base_intensity,ms1_base_rate,ms1_mol_rate
5,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12266_818.46.csv,1,826.2,494000.0,1,1813332.750,0,1321181.9,0,1321181.90,1.372508,1.372508e+00
6,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12311_821.59.csv,1,826.2,494000.0,0,1797438.750,1,1345650.5,1,1345650.50,1.335740,1.335740e+00
18,700_Amide,S98,576-42-1,10,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE10_neg_MS2_12301_818.49.csv,1,826.2,494000.0,1,1725657.875,0,1019900.4,0,1019900.40,1.691987,1.691987e+00
20,700_Amide,S98,576-42-1,10,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE10_neg_MS2_12391_824.7.csv,1,826.2,494000.0,0,1546713.875,1,1024069.9,1,1024069.90,1.510360,1.510360e+00
30,700_Amide,S98,576-42-1,100,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE100_neg_MS2_12196_811.45.csv,1,826.2,494000.0,0,1379039.500,0,-1.0,1,85472.31,16.134342,-1.379040e+06


In [385]:
concat_info_all_df['filepath'] = f'{out_root_path}/'+concat_info_all_df['batch']+'/NCE_compound/'+concat_info_all_df['sampleid'].astype(str)+'/'+concat_info_all_df['CAS No.']+'_NCE'+concat_info_all_df['EV'].astype(str)+'_'+concat_info_all_df['pos_neg']+'/'+concat_info_all_df['filename']
concat_info_all_df['bscp']=concat_info_all_df['batch']+'+'+concat_info_all_df['sampleid'].astype(str)+'+'+concat_info_all_df['CAS No.']+'+'+concat_info_all_df['pos_neg']
concat_info_all_df['bscep']=concat_info_all_df['batch']+'+'+concat_info_all_df['sampleid'].astype(str)+'+'+concat_info_all_df['CAS No.']+'+'+concat_info_all_df['EV'].astype(str)+'+'+concat_info_all_df['pos_neg']
concat_info_all_df['batch_sampleid']=concat_info_all_df['batch']+'/'+concat_info_all_df['sampleid'].astype(str)
print(concat_info_all_df.shape)
concat_info_all_df.head()

(204910, 23)


,batch,sampleid,CAS No.,EV,pos_neg,mz,Product Name,filename,refer_valid,ref_rt,...,is_max_mol,mol_intensity,is_max_base,base_intensity,ms1_base_rate,ms1_mol_rate,filepath,bscp,bscep,batch_sampleid
5,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12266_818.46.csv,1,826.2,...,0,1321181.9,0,1321181.90,1.372508,1.372508e+00,../../Fulaoshi/230821_mzML_res/rerun240410/700...,700_Amide+S98+576-42-1+neg,700_Amide+S98+576-42-1+1+neg,700_Amide/S98
6,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12311_821.59.csv,1,826.2,...,1,1345650.5,1,1345650.50,1.335740,1.335740e+00,../../Fulaoshi/230821_mzML_res/rerun240410/700...,700_Amide+S98+576-42-1+neg,700_Amide+S98+576-42-1+1+neg,700_Amide/S98
18,700_Amide,S98,576-42-1,10,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE10_neg_MS2_12301_818.49.csv,1,826.2,...,0,1019900.4,0,1019900.40,1.691987,1.691987e+00,../../Fulaoshi/230821_mzML_res/rerun240410/700...,700_Amide+S98+576-42-1+neg,700_Amide+S98+576-42-1+10+neg,700_Amide/S98
20,700_Amide,S98,576-42-1,10,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE10_neg_MS2_12391_824.7.csv,1,826.2,...,1,1024069.9,1,1024069.90,1.510360,1.510360e+00,../../Fulaoshi/230821_mzML_res/rerun240410/700...,700_Amide+S98+576-42-1+neg,700_Amide+S98+576-42-1+10+neg,700_Amide/S98
30,700_Amide,S98,576-42-1,100,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE100_neg_MS2_12196_811.45.csv,1,826.2,...,0,-1.0,1,85472.31,16.134342,-1.379040e+06,../../Fulaoshi/230821_mzML_res/rerun240410/700...,700_Amide+S98+576-42-1+neg,700_Amide+S98+576-42-1+100+neg,700_Amide/S98


In [362]:
bscp_list = concat_info_all_df['bscp'].unique().tolist()
len(bscp_list)

13642

In [230]:
batch_sampleid_list = concat_info_all_df['batch_sampleid'].unique().tolist()
len(batch_sampleid_list)

348

# Calculate fragments for the 13 energy values and output to file

In [386]:
def get_mz_count(temp13_df):
    ms2_df_list = []
    for i in temp13_df.index:
        temp13_df['filename']=temp13_df['filename'].astype(str)
        filename, filepath = temp13_df.loc[i,['filename','filepath']]
        if filename=='False':
            continue
        one_ms2_df = pd.read_csv(filepath,index_col=0)
        ms2_df_list.append(one_ms2_df)
    
    if ms2_df_list!=[]:
        con_ms2_df = pd.concat(ms2_df_list)
        con_ms2_df = con_ms2_df.reset_index(drop=True)

        mz13_count_dict = {}
        for i in con_ms2_df.index:
            mz = con_ms2_df.loc[i,['Mass']].tolist()[0]
            low,high = mzbatch.get_mz_range(mz)
            nums = con_ms2_df[(con_ms2_df['Mass']>=low) & (con_ms2_df['Mass']<=high)].shape[0]
            mz13_count_dict[mz]=nums
        return mz13_count_dict, len(ms2_df_list)
    else:
        return {}, 0

In [ ]:
n=0
for bscp in bscp_list[0:1]:
    fragment13_df = pd.DataFrame(columns=['batch', 'sampleid', 'CAS No.', 'pos_neg', 'mz', 'Product Name'
                                      , 'file_nums','ev_nums', 'mz13_count_dict', 'mz13_count1_list'])
    batch, sampleid, cas, pos_neg = bscp.split('+')
    temp13_df = concat_info_all_df[(concat_info_all_df['batch']==batch) & (concat_info_all_df['sampleid'].astype(str)==sampleid) 
                                & (concat_info_all_df['CAS No.']==cas) & (concat_info_all_df['pos_neg']==pos_neg)]
    ev_nums = temp13_df['EV'].unique().shape[0] 
    mz, product_name = temp13_df.iloc[0,[5,6]]
    mz13_count_dict, file_nums = get_mz_count(temp13_df)
    if mz13_count_dict=={}:
        continue
    if ev_nums == 13:
        mz13_count1_list = [key for key, value in mz13_count_dict.items() if value > 1]
        mz_only1_count_dict,file_nums_only1 = get_mz_count(temp13_df[temp13_df["EV"]==1])
        mz13_only1_count1_list = list(mz_only1_count_dict.keys())
        mz_only100_count_dict,file_nums_only100 = get_mz_count(temp13_df[temp13_df["EV"]==100])
        mz13_only100_count1_list = list(mz_only100_count_dict.keys())
        mz13_count1_list = list(set(mz13_count1_list+mz13_only1_count1_list+mz13_only100_count1_list))
    else:
        mz13_count1_list = list(mz13_count_dict.keys())
    mz13_count1_list = sorted(mz13_count1_list)
    fragment13_df.loc[n] = [batch, sampleid, cas, pos_neg, mz, product_name, file_nums, ev_nums, mz13_count_dict, mz13_count1_list]
    n+=1
    
    fragment_batch_path = f'{out_root_path}/{batch}/NCE_fragment13'
    if not os.path.exists(fragment_batch_path):
        os.makedirs(fragment_batch_path)
    while True:
        if os.path.exists(fragment_batch_path):
            break
    fragment13_df.to_csv(f'{fragment_batch_path}/{bscp}.csv', encoding='utf-8-sig')
    
    break

In [276]:
mz_only1_count_dict,file_nums_only1 = get_mz_count(temp13_df[temp13_df["EV"]==1])
mz_only100_count_dict,file_nums_only100 = get_mz_count(temp13_df[temp13_df["EV"]==100])
mz13_only1_count1_list = list(mz_only1_count_dict.keys())
mz13_only100_count1_list = list(mz_only100_count_dict.keys())

### Check what doesn't by referring to the file

In [345]:
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all.xlsx",sheet_name="concat",index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch']+'/'+ref_all_df['sampleid']
ref_all_df['bscp']=ref_all_df['batch']+'+'+ref_all_df['sampleid'].astype(str)+'+'+ref_all_df['CAS No.']+'+'+ref_all_df['pos_neg']
ref_all_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,batch_sampleid
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos,873_Amide/26386
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg,873_Amide/26386
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos,873_Amide/26386
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg,873_Amide/26386
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,neg,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg,873_Amide/26386


In [351]:
bscp_list = ref_all_df["bscp"].unique().tolist()
len(bscp_list)

13668

In [352]:
con_bscp_list = concat_info_all_df['bscp'].unique().tolist()

In [353]:
unique_to_bscp_list = [item for item in bscp_list if item not in con_bscp_list]
print(len(bscp_list), len(con_bscp_list), len(unique_to_bscp_list))

13668 13642 26


In [354]:
unique_to_con_bscp_list = [item for item in con_bscp_list if item not in bscp_list]
print(len(bscp_list), len(con_bscp_list), len(unique_to_con_bscp_list))

13668 13642 0


In [357]:
unique_to_bscp_list[0], bscp_list.index(unique_to_bscp_list[0])

('873_Amide+26386+1477-68-5+neg', 60)

# Merge all data

### Merge data for 13 energies

In [376]:
for batch in batch_list:
    fragment13_root_path = f'{out_root_path}/{batch}/NCE_fragment13' 
    fragment13_df_list = []

    fragment13_filename_list = os.listdir(fragment13_root_path)
    for fragment13_filename in fragment13_filename_list:
        one_fragment13_df = pd.read_csv(f'{fragment13_root_path}/{fragment13_filename}',index_col=0)
        fragment13_df_list.append(one_fragment13_df)

    fragment13_all_df = pd.concat(fragment13_df_list)
    fragment13_all_df.to_csv(f'{out_root_path}/{batch}/fragment13_all_df.csv')


### Merge batch data

In [378]:
batch_fragment13_df_list = []
batch_fragment1_df_list = []

for batch in batch_list:
    
    batch_fragment13_df = pd.read_csv(f'{out_root_path}/{batch}/fragment13_all_df.csv', index_col=0)
    batch_fragment13_df_list.append(batch_fragment13_df)
    
con_fragment13_df = pd.concat(batch_fragment13_df_list)
con_fragment13_df.to_csv(f'{out_root_path}/con_fragment13_df.csv')



# Merge the two filtered mz_list

In [194]:
import pandas as pd
import ast,os
import mzbatch
import numpy as np

# Set path

In [229]:
root_path = '../../Fulaoshi/230821_mzML'
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

In [230]:
smi_ref_df = pd.read_excel(f'{root_path}/smi_ref_df.xlsx',index_col=0)
smi_ref_df['bscp']=smi_ref_df['batch']+'+'+smi_ref_df['sampleid'].astype(str)+'+'+smi_ref_df['CAS No.']+'+'+smi_ref_df['pos_neg']
bscp_list = smi_ref_df['bscp'].unique().tolist()
print(smi_ref_df.shape)
smi_ref_df.tail()

(13668, 14)


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,MW,pos_neg,is_valid,mz,ref_rt,NL,bscp,batch_sampleid
13663,700_C18,C18,S104,L-Allothreonine,28954-12-3,C4H9NO3,119.12,neg,1,118.05096,46.8,9.35E5,700_C18+S104+28954-12-3+neg,[M-H]-
13664,700_C18,C18,S104,3-O-Ethyl-L-ascorbic acid,86404-04-8,C8H12O6,204.18,neg,1,203.05611,307.8,9.02E5,700_C18+S104+86404-04-8+neg,[M-H]-
13665,700_C18,C18,S104,n6-(Aminoiminomethyl)lysine,yhh0927-1,C7H16N4O2,188.23,pos,1,189.13460,45.6,9.94E6,700_C18+S104+yhh0927-1+pos,[M+H]+
13666,700_C18,C18,S104,Galactosamine hydrochloride,1886979-58-3,C6H14ClNO5,215.63,pos,1,180.08664,43.8,2.15E7,700_C18+S104+1886979-58-3+pos,[M+H]+
13667,700_C18,C18,S104,L-Cysteine hydrochloride monohydrate,7048-04-6,C3H10ClNO3S,175.64,neg,1,120.01247,45.0,6.20E6,700_C18+S104+7048-04-6+neg,[M-H]-


In [232]:
con_fragment13_df = pd.read_csv(f'{out_root_path}/con_fragment13_df.csv',index_col=0)
print(con_fragment13_df.shape)
con_fragment13_df.head(1)

(13642, 10)


,batch,sampleid,CAS No.,pos_neg,mz,Product Name,file_nums,ev_nums,mz13_count_dict,mz13_count1_list
0,700_Amide,S98,576-42-1,neg,209.03029,D-Glucaric acid (potassium),24,13,"{60.346050603148434: 1, 65.56717364328462: 1, ...","[46.78787664009276, 47.98073029899072, 49.0779..."


In [199]:
con_fragment13_df["mz13_count1_list"]

0      [46.78787664009276, 47.98073029899072, 49.0779...
0      [59.30694548757083, 61.008287913769664, 63.023...
1      [51.95411781404511, 55.65065645608881, 56.7898...
1      [65.03856104965429, 65.038658436575, 65.038741...
2      [50.03290768526243, 50.46216240915907, 51.0230...
                             ...                        
437    [53.00342654836226, 55.01893332440959, 55.0189...
438    [87.00824423248562, 87.00872435089443, 87.0087...
439    [56.4433289598851, 59.013749678915474, 59.0137...
440    [53.27499476037805, 55.02994927928626, 55.0301...
441    [40.34946568389486, 41.00328926857089, 41.0033...
Name: mz13_count1_list, Length: 13642, dtype: object

In [200]:
con_fragment13_df[con_fragment13_df["mz13_count1_list"]=="[]"]

,batch,sampleid,CAS No.,pos_neg,mz,Product Name,file_nums,ev_nums,mz13_count_dict,mz13_count1_list


- Check if temp_con_fragment13_df is empty, indicating no files were found

In [202]:
bscp_list[0], len(bscp_list)

('873_Amide+26386+1867-73-8+pos', 13668)

In [1]:
temp_outpath

[[999.99, 1000.01], [1999.98, 2000.02], [2999.97, 3000.03]]

In [220]:
for bscp in bscp_list[844:845]:
    print(bscp)
    batch, sampleid, cas, pos_neg = bscp.split("+")
    temp_df = smi_ref_df[smi_ref_df['bscp']==bscp].copy()
    
    try:
        cal_mz_df = pd.read_csv(f'output/smi_ref_df/{bscp}.csv',index_col=0)
        fragment13_df = pd.read_csv(f"{out_root_path}/{batch}/NCE_fragment13/{bscp}.csv",index_col=0)
    except FileNotFoundError:
        print("FileNotFound:",f"{out_root_path}/{batch}/NCE_fragment13/{bscp}.csv")
        continue
    cal_mzlist = eval(cal_mz_df.iloc[0,-1])
    fragment13_mzlist = eval(fragment13_df.iloc[0,-1])
    
    status_list = []
    for j in fragment13_mzlist:
        low,up = mzbatch.get_mz_range(j)
        temp_compare_df = pd.DataFrame({'cal_fragment':cal_mzlist})
        if temp_compare_df[(temp_compare_df['cal_fragment']>=low) & (temp_compare_df['cal_fragment']<=up)].shape[0]>0:
            status_list.append(True)
        else:
            status_list.append(False)  

    temp_valid_df = pd.DataFrame({'fragment':fragment13_mzlist,'is_valid':status_list})
    fragment_valid_list = temp_valid_df[temp_valid_df['is_valid']]['fragment'].to_list()
    fragment_valid_list = sorted(fragment_valid_list)
    
    temp_df['filter_fragment'] = [fragment_valid_list]
    temp_outpath = f"{out_root_path}/{batch}/NCE_filter_fragment/"
    if not os.path.exists(temp_outpath):
        os.makedirs(temp_outpath)
    while True:
        if os.path.exists(temp_outpath):
            break
    temp_df.to_csv(f"{temp_outpath}/{bscp}.csv")
    break
    

873_Amide+2_2+2450-53-5+pos


# Merge files

In [226]:
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']

In [ ]:
for batch in batch_list:
    fragment13_root_path = f'{out_root_path}/{batch}/NCE_filter_fragment' 
    fragment13_df_list = []

    fragment13_filename_list = os.listdir(fragment13_root_path)
    for fragment13_filename in fragment13_filename_list:
        one_fragment13_df = pd.read_csv(f'{fragment13_root_path}/{fragment13_filename}',index_col=0)
        fragment13_df_list.append(one_fragment13_df)

    fragment13_all_df = pd.concat(fragment13_df_list)
    fragment13_all_df.to_csv(f'{out_root_path}/{batch}/filter_fragment_df.csv')

In [228]:
batch_filter_fragment_df_list = []

for batch in batch_list:
    
    batch_filter_fragment_df = pd.read_csv(f'{out_root_path}/{batch}/filter_fragment_df.csv', index_col=0)
    batch_filter_fragment_df_list.append(batch_filter_fragment_df)
    
con_filter_fragment_df = pd.concat(batch_filter_fragment_df_list)
con_filter_fragment_df.to_csv(f'{out_root_path}/con_filter_fragment_df.csv')
